In [1]:
import barzur20aft
import compiler
import numpy as np
import pandas

In [2]:
def bitcoin_mdp(*args, **kwargs):
    model = barzur20aft.Bitcoin(*args, **kwargs)
    c = compiler.Compiler(model)
    return c.mdp()

In [3]:
alpha = [0.01, 0.05, 0.1, 0.2, 0.3, 0.35, 0.4, 0.45, 0.49]
ptmdp = []
mdp = []
for a in alpha:
    print(a)
    m = bitcoin_mdp(alpha=a, gamma=0, maximum_fork_length=25)
    mdp.append(m)
    ptmdp.append(barzur20aft.ptmdp(m, horizon=100))

0.01
0.05
0.1
0.2
0.3
0.35
0.4
0.45
0.49


In [4]:
mdp[0]

MDP of size 1624 / 4 / 6318 / 3.9

In [5]:
vi = []
for i, m in enumerate(ptmdp):
    res = m.value_iteration(value_eps=0.01)
    vi.append(res)
    v = res["vi_value"]
    p = res["vi_policy"]
    iter = res["vi_iter"]
    print(
        f"alpha={alpha[i]:.2f} iter={iter:3d} v[0]={v[0]:.2f} p={p[:10]} {sum(p)} {hash(p.tobytes()) % 10000}"
    )

alpha=0.01 iter= 30 v[0]=0.14 p=[0 2 1 0 0 0 1 0 0 2] 1370 1648
alpha=0.05 iter=315 v[0]=3.97 p=[0 2 1 0 0 0 1 0 0 2] 1386 7653
alpha=0.10 iter=439 v[0]=8.89 p=[0 2 1 0 0 0 1 0 0 2] 1380 5994
alpha=0.20 iter=535 v[0]=18.64 p=[0 2 1 0 0 0 1 0 0 2] 1350 2072
alpha=0.30 iter=541 v[0]=28.02 p=[0 2 1 0 0 0 1 0 0 2] 1302 2294
alpha=0.35 iter=571 v[0]=35.46 p=[0 0 1 0 0 0 1 0 0 2] 1282 9694
alpha=0.40 iter=628 v[0]=47.30 p=[0 0 1 0 0 0 1 0 0 2] 1260 7377
alpha=0.45 iter=689 v[0]=64.32 p=[0 0 1 0 0 0 1 0 0 0] 1236 9802
alpha=0.49 iter=747 v[0]=80.97 p=[0 0 1 0 0 0 1 0 0 0] 1200 3043


In [6]:
def rows():
    for i, a in enumerate(alpha):
        rpp = mdp[i].reward_per_progress(vi[i]["vi_policy"], eps=0.0001)
        yield dict(alpha=a) | rpp


pandas.DataFrame(rows())

,alpha,ss_n,ss_nonzero,ss_time,rpp,rpp_iter,rpp_time
0,0.01,4,4,0.001276,0.010000,2,0.003981
1,0.05,4,4,0.000790,0.050000,2,0.003770
2,0.10,4,4,0.000790,0.100000,2,0.003746
3,0.20,4,4,0.000781,0.200000,2,0.003764
4,0.30,4,4,0.000762,0.300000,2,0.003787
5,0.35,716,716,0.127626,0.370615,2,0.395020
6,0.40,738,738,0.116256,0.487055,2,0.404879
7,0.45,758,758,0.092692,0.652790,2,0.402160
8,0.49,790,790,0.114792,0.828411,2,0.405096
